In [3]:
print("hello")

hello


## Utils Functions

In [4]:
def text_to_binary(text):
    # Mengubah setiap karakter menjadi representasi biner 8-bit
    binary_representation = ' '.join(format(ord(char), '08b') for char in text)
    return binary_representation




def binary_to_text(binary_string):
    # Membagi biner menjadi grup 8 bit
    chars = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]

    # Mengubah setiap grup biner menjadi karakter
    text = ''.join([chr(int(char, 2)) for char in chars])

    return text

def binary_to_hex(binary_string):
    # Pastikan panjang string biner adalah kelipatan 4
    binary_string = binary_string.zfill(len(binary_string) + (4 - len(binary_string) % 4) % 4)

    # Konversi biner ke integer, lalu integer ke heksadesimal
    hex_string = hex(int(binary_string, 2))[2:]  # [2:] untuk menghilangkan prefix '0x'

    return hex_string.upper()


def rm_ws(text):
    # Menghilangkan whitespace
    return text.replace(" ", "")


In [5]:
# Contoh penggunaan
kata = "fasilkom"
binary_kata = text_to_binary(kata)
print(binary_kata)

# Contoh penggunaan
binary_string = rm_ws('01100110 01100001 01110011 01101001 01101100 01101011 01101111 01101101')  # "Hello" dalam biner
text = binary_to_text(binary_string)
print(f"Teks: {text}")


# Contoh penggunaan
kata = "46114220"
binary_kata = text_to_binary(kata)
print(binary_kata)

# Contoh penggunaan
binary_string = rm_ws('00110100 00110110 00110001 00110001 00110100 00110010 00110010 00110000') # "Hello" dalam biner
text = binary_to_text(binary_string)
print(f"Teks: {text}")

01100110 01100001 01110011 01101001 01101100 01101011 01101111 01101101
Teks: fasilkom
00110100 00110110 00110001 00110001 00110100 00110010 00110010 00110000
Teks: 46114220


## Encrypt

In [6]:
# Define the necessary DES tables (initial permutation, final permutation, etc.)
INITIAL_PERM = [58, 50, 42, 34, 26, 18, 10, 2,
                60, 52, 44, 36, 28, 20, 12, 4,
                62, 54, 46, 38, 30, 22, 14, 6,
                64, 56, 48, 40, 32, 24, 16, 8,
                57, 49, 41, 33, 25, 17, 9, 1,
                59, 51, 43, 35, 27, 19, 11, 3,
                61, 53, 45, 37, 29, 21, 13, 5,
                63, 55, 47, 39, 31, 23, 15, 7]

FINAL_PERM = [40, 8, 48, 16, 56, 24, 64, 32,
              39, 7, 47, 15, 55, 23, 63, 31,
              38, 6, 46, 14, 54, 22, 62, 30,
              37, 5, 45, 13, 53, 21, 61, 29,
              36, 4, 44, 12, 52, 20, 60, 28,
              35, 3, 43, 11, 51, 19, 59, 27,
              34, 2, 42, 10, 50, 18, 58, 26,
              33, 1, 41, 9, 49, 17, 57, 25]

EXPAND_TABLE = [32, 1, 2, 3, 4, 5,
                4, 5, 6, 7, 8, 9,
                8, 9, 10, 11, 12, 13,
                12, 13, 14, 15, 16, 17,
                16, 17, 18, 19, 20, 21,
                20, 21, 22, 23, 24, 25,
                24, 25, 26, 27, 28, 29,
                28, 29, 30, 31, 32, 1]

# Substitute boxes for DES algorithm (8 different S-boxes)
SUB_BOXES = [
    # S1
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

    # S2
    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

    # S3
    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

    # S4
    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

    # S5
    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

    # S6
    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

    # S7
    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

    # S8
    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
]

# Permutation after applying S-boxes
PERM_TABLE = [16, 7, 20, 21, 29, 12, 28, 17,
              1, 15, 23, 26, 5, 18, 31, 10,
              2, 8, 24, 14, 32, 27, 3, 9,
              19, 13, 30, 6, 22, 11, 4, 25]

# Key Permutation tables
PC1 = [57, 49, 41, 33, 25, 17, 9,
       1, 58, 50, 42, 34, 26, 18,
       10, 2, 59, 51, 43, 35, 27,
       19, 11, 3, 60, 52, 44, 36,
       63, 55, 47, 39, 31, 23, 15,
       7, 62, 54, 46, 38, 30, 22,
       14, 6, 61, 53, 45, 37, 29,
       21, 13, 5, 28, 20, 12, 4]

SHIFT_AMOUNTS = [1, 1, 2, 2, 2, 2, 2, 2,
                 1, 2, 2, 2, 2, 2, 2, 1]

PC2 = [14, 17, 11, 24, 1, 5,
       3, 28, 15, 6, 21, 10,
       23, 19, 12, 4, 26, 8,
       16, 7, 27, 20, 13, 2,
       41, 52, 31, 37, 47, 55,
       30, 40, 51, 45, 33, 48,
       44, 49, 39, 56, 34, 53,
       46, 42, 50, 36, 29, 32]

# Helper functions for DES operations
def permute_bits(block, table):
    return ''.join([block[i - 1] for i in table])

def xor_bits(a, b):
    return ''.join(['0' if i == j else '1' for i, j in zip(a, b)])

def sbox_substitution(bits, sbox_idx):
    sbox = SUB_BOXES[sbox_idx]
    row = int(bits[0] + bits[5], 2)
    col = int(bits[1:5], 2)
    return format(sbox[row][col], '04b')

def split_in_half(data):
    half = len(data) // 2
    return data[:half], data[half:]

def group_binary_string(binary_str, group_size):
    return ' '.join([binary_str[i:i + group_size] for i in range(0, len(binary_str), group_size)])

# Key schedule to generate round keys
def generate_keys(key_input):
    key = permute_bits(key_input, PC1)
    C_part, D_part = split_in_half(key)
    round_keys = []
    print(f"Initial C: {group_binary_string(C_part, 8)}, D: {group_binary_string(D_part, 8)}")
    
    for round_num in range(16):
        C_part = C_part[SHIFT_AMOUNTS[round_num]:] + C_part[:SHIFT_AMOUNTS[round_num]]
        D_part = D_part[SHIFT_AMOUNTS[round_num]:] + D_part[:SHIFT_AMOUNTS[round_num]]
        round_key = permute_bits(C_part + D_part, PC2)
        round_keys.append(round_key)
        print(f"Round {round_num + 1}: C: {group_binary_string(C_part, 8)}, D: {group_binary_string(D_part, 8)}, Round Key: {group_binary_string(round_key, 6)}")
    
    return round_keys

# DES function for 16 rounds
def des_function(R_half, key_round):
    expanded_R = permute_bits(R_half, EXPAND_TABLE)
    print(f"Expanded: {group_binary_string(expanded_R, 6)}")
    print(f"Key for Round: {group_binary_string(key_round, 6)}")
    
    # XOR step
    result_xor = xor_bits(expanded_R, key_round)
    print(f"XOR Result: {group_binary_string(result_xor, 6)}")
    
    # S-box transformation
    sbox_result = ''
    for box_num in range(8):
        six_bit_block = result_xor[box_num * 6: (box_num + 1) * 6]
        sbox_result += sbox_substitution(six_bit_block, box_num)
    
    print(f"S-box Result: {group_binary_string(sbox_result, 4)}")
    
    # Final permutation for this round
    return permute_bits(sbox_result, PERM_TABLE)

# DES encryption
def des_encrypt(data_input, key_input):
    # Initial permutation
    permuted_data = permute_bits(data_input, INITIAL_PERM)
    L_half, R_half = split_in_half(permuted_data)
    
    # Generate round keys
    keys = generate_keys(key_input)
    
    # 16 rounds
    for round_idx, round_key in enumerate(keys):
        print("")
        print(f"Round {round_idx + 1} Input: L={group_binary_string(L_half, 4)}, R={group_binary_string(R_half, 4)}")
        
        # DES function for this round
        round_output = des_function(R_half, round_key)
        new_R_half = xor_bits(L_half, round_output)
        L_half = R_half
        R_half = new_R_half
    
    # Final permutation
    final_result = permute_bits(R_half + L_half, FINAL_PERM)
    return final_result


In [7]:

# Example usage
plaintext = rm_ws('01100110 01100001 01110011 01101001 01101100 01101011 01101111 01101101')  # 64-bit input
key = rm_ws('00110100 00110110 00110001 00110001 00110100 00110010 00110010 00110000')  # 64-bit key


# Encrypt and print the ciphertext
ciphertext = des_encrypt(plaintext, key)
print(f"\nCiphertext: {ciphertext}")
print(f"\nCiphertext: {ciphertext}")

Initial C: 00000000 00000000 11111111 1111, D: 01100010 00010011 00000000 1111
Round 1: C: 00000000 00000001 11111111 1110, D: 11000100 00100110 00000001 1110, Round Key: 010100 000010 110010 101100 010001 100000 001011 010010
Round 2: C: 00000000 00000011 11111111 1100, D: 10001000 01001100 00000011 1101, Round Key: 010100 001010 110010 100100 110000 001010 000101 010010
Round 3: C: 00000000 00001111 11111111 0000, D: 00100001 00110000 00001111 0110, Round Key: 110100 001010 110000 100110 011001 011000 011000 001100
Round 4: C: 00000000 00111111 11111100 0000, D: 10000100 11000000 00111101 1000, Round Key: 111000 001010 011000 100110 010110 000001 010011 001010
Round 5: C: 00000000 11111111 11110000 0000, D: 00010011 00000000 11110110 0010, Round Key: 111000 001001 011000 100110 000011 001101 000000 101101
Round 6: C: 00000011 11111111 11000000 0000, D: 01001100 00000011 11011000 1000, Round Key: 111000 001001 001001 110010 000000 100111 110011 100000
Round 7: C: 00001111 11111111 000

## Decrypt

In [8]:
# Step-by-step DES decryption process
def des_decrypt(cipher_text, decryption_key):
    # Initial permutation
    permuted_text = permute_bits(cipher_text, INITIAL_PERM)
    left_half, right_half = split_in_half(permuted_text)

    # Generate 16 round keys (in reverse order for decryption)
    round_keys = generate_keys(decryption_key)

    # 16 rounds of DES decryption (keys used in reverse)
    for round_num in range(15, -1, -1):
        print(f"\nRound {round_num + 1}:")
        temp_right = right_half
        right_half = xor_bits(left_half, des_function(right_half, round_keys[round_num]))
        left_half = temp_right

        print(f"L{round_num + 1} = {group_binary_string(left_half, 8)}, R{round_num + 1} = {group_binary_string(right_half, 8)}")

    # Final permutation
    final_result = permute_bits(right_half + left_half, FINAL_PERM)

    return group_binary_string(final_result, 8)

In [9]:
# Example ciphertext and key (in binary)
cipher_text = "0001010001000111000001111101100111011010100001111001110101110001"
key = rm_ws('00110100 00110110 00110001 00110001 00110100 00110010 00110010 00110000')  # 64-bit key

# Decrypt and display the plaintext
decrypted_plaintext = des_decrypt(cipher_text, key)
print(f"\nDecrypted Plaintext: {decrypted_plaintext}")

Initial C: 00000000 00000000 11111111 1111, D: 01100010 00010011 00000000 1111
Round 1: C: 00000000 00000001 11111111 1110, D: 11000100 00100110 00000001 1110, Round Key: 010100 000010 110010 101100 010001 100000 001011 010010
Round 2: C: 00000000 00000011 11111111 1100, D: 10001000 01001100 00000011 1101, Round Key: 010100 001010 110010 100100 110000 001010 000101 010010
Round 3: C: 00000000 00001111 11111111 0000, D: 00100001 00110000 00001111 0110, Round Key: 110100 001010 110000 100110 011001 011000 011000 001100
Round 4: C: 00000000 00111111 11111100 0000, D: 10000100 11000000 00111101 1000, Round Key: 111000 001010 011000 100110 010110 000001 010011 001010
Round 5: C: 00000000 11111111 11110000 0000, D: 00010011 00000000 11110110 0010, Round Key: 111000 001001 011000 100110 000011 001101 000000 101101
Round 6: C: 00000011 11111111 11000000 0000, D: 01001100 00000011 11011000 1000, Round Key: 111000 001001 001001 110010 000000 100111 110011 100000
Round 7: C: 00001111 11111111 000

In [10]:
text = binary_to_text(rm_ws("01100110 01100001 01110011 01101001 01101100 01101011 01101111 01101101"))

print(f"Teks: {text}")

Teks: fasilkom
